# Extractive Summarization

In [1]:
import os
# variable we need to set to get around macOS related issue
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
from summarizer import Summarizer

In [41]:
with open('./transcripts/joe_biden.txt', 'r') as file:
    data = file.read()
data_arr = data.split("\n")
print(len(data_arr))
#print(data_arr[0])
cleaned_data = ""
for i in range(1, len(data_arr)):
    cleaned_data += data_arr[i]
print(cleaned_data)

429
This is America’s day.This is democracy’s day.A day of history and hope.Of renewal and resolve.Through a crucible for the ages America has been tested anew and America has risen to the challenge.Today, we celebrate the triumph not of a candidate, but of a cause, the cause of democracy.The will of the people has been heard and the will of the people has been heeded.We have learned again that democracy is precious.Democracy is fragile.And at this hour, my friends, democracy has prevailed.So now, on this hallowed ground where just days ago violence sought to shake this Capitol’s very foundation, we come together as one nation, under God, indivisible, to carry out the peaceful transfer of power as we have for more than two centuries.We look ahead in our uniquely American way – restless, bold, optimistic – and set our sights on the nation we know we can be and we must be.I thank my predecessors of both parties for their presence here.I thank them from the bottom of my heart.You know the

In [42]:
preprocess_text = cleaned_data.strip().replace("\n","")

In [52]:
extractive_model = Summarizer()
extractive_result = extractive_model(preprocess_text, ratio=0.2)

In [53]:
extractive_result

'Through a crucible for the ages America has been tested anew and America has risen to the challenge. And at this hour, my friends, democracy has prevailed. I thank my predecessors of both parties for their presence here. Hundreds of thousands of businesses closed. The dream of justice for all will be deferred no longer. A cry for survival comes from the planet itself. Today, on this January day, my whole soul is in this:Bringing America together. We can teach our children in safe schools. We can make America, once again, the leading force for good in the world. History, faith, and reason show the way, the way of unity. And so today, at this time and in this place, let us start afresh. Every disagreement doesn’t have to be a cause for total war. Here we stand, in the shadow of a Capitol dome that was completed amid the Civil War, when the Union itself hung in the balance. Here we stand across the Potomac from Arlington National Cemetery, where heroes who gave the last full measure of d

In [54]:
array_result = extractive_result.split(' ')
print(len(array_result))

351


# Abstractive Summarization

In [3]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

In [58]:
t5_prepared_Text = "summarize: "+extractive_result
print ("original text preprocessed: \n", extractive_result)

original text preprocessed: 
 Through a crucible for the ages America has been tested anew and America has risen to the challenge. And at this hour, my friends, democracy has prevailed. I thank my predecessors of both parties for their presence here. Hundreds of thousands of businesses closed. The dream of justice for all will be deferred no longer. A cry for survival comes from the planet itself. Today, on this January day, my whole soul is in this:Bringing America together. We can teach our children in safe schools. We can make America, once again, the leading force for good in the world. History, faith, and reason show the way, the way of unity. And so today, at this time and in this place, let us start afresh. Every disagreement doesn’t have to be a cause for total war. Here we stand, in the shadow of a Capitol dome that was completed amid the Civil War, when the Union itself hung in the balance. Here we stand across the Potomac from Arlington National Cemetery, where heroes who ga

In [59]:
tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=60,
                                    max_length=300,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)



Summarized text: 
 today, on this January day, my whole soul is in this:Bringing America together. we can make America, once again, the leading force for good in the world, history, faith, and reason show the way of unity. and so today let us start afresh. every disagreement doesn’t have to be causing total war. here we stand across the Potomac from Arlington National Cemetery. heroes who gave the last full measure of devotion rest in eternal peace.


In [32]:
tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=6,
                                    no_repeat_ngram_size=2,
                                    min_length=60,
                                    max_length=300,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)

Token indices sequence length is longer than the specified maximum sequence length for this model (545 > 512). Running this sequence through the model will result in indexing errors




Summarized text: 
 we've sold enough new work and I'll hit some wins in a minute. this week we had wins at 11 or 12 different clients, but we have excited to announce 11 in terms who accepted the non paid internship and will start on June 15th. be 100% virtual experience for them but once they do get on board will look to make them feel welcome.


## Cleaned Transcript Processing

In [18]:
from os import listdir
import os
from summarizer import Summarizer
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

# variable we need to set to get around 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')
transcript_folder = './cleaned_transcripts/'

In [19]:
def get_extractive_summary(cleaned_data, extracted_ratio):
    preprocess_text = cleaned_data.strip().replace("\n","")
    extractive_model = Summarizer()
    return extractive_model(preprocess_text, ratio=extracted_ratio)
    

In [20]:
def get_extract_ratio(target, word_count):
    return target / word_count

In [21]:
def get_abstractive_summary(input_text):
    t5_prepared_Text = "summarize: "+input_text
    tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
    # summmarize 
    summary_ids = model.generate(tokenized_text,
                                        num_beams=6,
                                        no_repeat_ngram_size=2,
                                        min_length=90,
                                        max_length=300,
                                        early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [22]:
files = listdir(transcript_folder)
print(files)

['20200814_cleaned.txt', '20200424_cleaned.txt', '20201106_Cleaned.txt', '20200717_cleaned.txt', '20200515_cleaned.txt']


In [6]:
for file in files:
    file_path = transcript_folder + file
    print(file_path)
    with open(file_path, 'r') as file_reader:
        data = file_reader.read()
    data_arr = data.split("\n")
    cleaned_data = ""
    for i in range(1, len(data_arr)):
        cleaned_data += data_arr[i]
    word_count = len(cleaned_data.split(" "))
    ratio = get_extract_ratio(450.0, word_count)
    extractive_summary = get_extractive_summary(cleaned_data, ratio)
    print("Extractive length: " + str(len(extractive_summary.split(" "))))
    abstractive_summary = get_abstractive_summary(extractive_summary)
    abstractive_word_count = len(abstractive_summary.split(" "))
    print("Abstractive length: " + str(abstractive_word_count))
    print("ABSTRACTIVE_SUMMARY")
    print(abstractive_summary)
    extractive_ratio = get_extract_ratio(abstractive_word_count * 1.1, word_count)
    extractive_short = get_extractive_summary(cleaned_data, extractive_ratio)
    short_ext_word_count = len(extractive_short.split(" "))
    print('\n')
    print("Short extractive word count: " + str(short_ext_word_count))
    print("EXTRACTIVE_SUMMARY")
    print(extractive_short)
    print('\n\n\n')

./cleaned_transcripts/20200814_cleaned.txt
Extractive length: 324
Abstractive length: 72
ABSTRACTIVE_SUMMARY
we'll be rolling out a new CPM tool in September, which should make the process your monthly check-ins with your career coach. no changes to the actual expectations, framework or the expectations this year, or any changes in the sort of promotion process or compensation process. the new tool will be rolled out in september, but it's slated to be released in january, and is expected to take place in early october.


Short extractive word count: 94
EXTRACTIVE_SUMMARY
A quick video this week just to give you an update on what's going on. When we had to make the adjustment to the internship program back in the spring, Ann, Meredith, and Evan in particular did an incredible job of organizing that and the parts that I participated in and they just set a whole agenda, it was just thoughtful an I'm sure it was tremendously valuable and impactful to the to those young people and so well 

Token indices sequence length is longer than the specified maximum sequence length for this model (549 > 512). Running this sequence through the model will result in indexing errors


Extractive length: 427
Abstractive length: 84
ABSTRACTIVE_SUMMARY
we're beginning to think about hiring again in a couple of key areas. we had wins at I think 11 or 12 different clients this week, but it was really good week in terms of utilization and ultimately you know close to our revenue target. i'm thankful for all of you grateful for the work that you continue to put in. and as I mentioned earlier, we are starting to begin to even consider hiring back, which is an awesome position to be in!


Short extractive word count: 143
EXTRACTIVE_SUMMARY
You know, at this point May is looking really good in terms of utilization and and ultimately you know close to our revenue target. Very, very positive sign for the business and I'm very thankful for that. Want to celebrate some of the Impact awards and the Junkyard Dog Award. From a Junkyard Dog award standpoint this this week it's going to go to Anna Grace Franklin, who on Earth of Fix and add Orthofix made herself so invaluable to the c

## Larger Model

In [23]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')
transcript_folder = './cleaned_transcripts/'

In [24]:
from os import listdir
files = listdir(transcript_folder)
print(files)

['20200814_cleaned.txt', '20200424_cleaned.txt', '20201106_Cleaned.txt', '20200717_cleaned.txt', '20200515_cleaned.txt']


In [25]:
def get_abstractive_summary(input_text):
    t5_prepared_Text = "summarize: "+input_text
    tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
    # summmarize 
    summary_ids = model.generate(tokenized_text,
                                        num_beams=6,
                                        no_repeat_ngram_size=2,
                                        min_length=90,
                                        max_length=300,
                                        early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [27]:
punctuation = [".", "-", "'", ","]
for file in files:
    file_path = transcript_folder + file
    print(file_path)
    with open(file_path, 'r') as file_reader:
        data = file_reader.read()
    data_arr = data.split("\n")
    cleaned_data = ""
    for i in range(1, len(data_arr)):
        cleaned_data += data_arr[i]
    truncated_data = ""
    punctuation_count = 0   
    for i in range(0, 425):
        if(i < len(cleaned_data)):
            word = cleaned_data.split(" ")[i]
            truncated_data += word
            truncated_data += " "
            for letter in word:
                if letter in punctuation:
                    punctuation_count += 1
        else:
            break
    word_count = len(truncated_data.split(" "))
    print(f"Punctuation count: {punctuation_count}")
    print(f"Word count: {word_count}")
    print("\n")
    abstractive_summary = get_abstractive_summary(truncated_data)
    abstractive_word_count = len(abstractive_summary.split(" "))
    print("Abstractive length: " + str(abstractive_word_count))
    print("ABSTRACTIVE_SUMMARY")
    print(abstractive_summary)

./cleaned_transcripts/20200814_cleaned.txt
Punctuation count: 38
Word count: 426


Abstractive length: 86
ABSTRACTIVE_SUMMARY
this week we hosted our intern Leadership Development Conference, which was part of what we were able to offer the interns that did the internship with us, as well as some of those who chose not to. it was thoughtful and impactful to the young people and so well done, and it really blew me away in terms of just its professionalism, the quality of the work, how many people contributed to it across certainly our marketing team and the UX team being the heavy lifters.
./cleaned_transcripts/20200424_cleaned.txt
Punctuation count: 32
Word count: 426


Abstractive length: 81
ABSTRACTIVE_SUMMARY
we're working on the Credera version of what reopening may look like. it's under underway and specifically the guiding principles that we are working from there is that first and foremost health and safety of our team is what we care the most about. if you have a job post gradu

Token indices sequence length is longer than the specified maximum sequence length for this model (556 > 512). Running this sequence through the model will result in indexing errors


Abstractive length: 74
ABSTRACTIVE_SUMMARY
this week we got pretty big wins at wild type in NRG, Echelon Fitness, teavana. but really big week there and really strong sales pipeline. looking forward because that's enabled us to do something that was really fun, which is to actually reach back out to all of our campus hires that are supposed to start this fall and offer them the opportunity if they wanted to begin either one or two months early.
./cleaned_transcripts/20200515_cleaned.txt
Punctuation count: 60
Word count: 426


Abstractive length: 73
ABSTRACTIVE_SUMMARY
we're just about one month away from when we expect to close that and bring them into the Credera family as well. this is a good new business win in any economic climates, but especially good in this current one that we’re in.Quick just kind of follow up on the campus and internship program so that they continue to receive good feedback from the universities that were impacted and then from our interns.
